# Append crimes
As mentioned in the article, we have two sources of information about crimes in Bogotá, Colombia. The first one is NUSE, which represents the crime reported by the citizenship, and the second one is SIEDCO, the official record of crimes of the Colombian Police Department. By comparing the different reports in SIEDCO and NUSE, it can be observed that many reports in NUSE do not appear in SIEDCO and vice-versa. 

The objective of this script is to append both datasets to consolidate all the violent crime information of Bogotá for 2018. Here, we applied the same algorithm to remove duplicated crimes that was used in `1_clean_NUSE.ipynb`. To do so, we perform the following steps:
1. Import `violent_nuse.csv` and `siedco_raw.csv`.
2. Filter `siedco_raw.csv` to have only violent crimes. The data frame with only this kind of crimes is called `violent_siedco`.
3. As the typology of the crimes is not in the same format in NUSE and SIEDCO, we created a dictionary to homogenize the typology of both sets. In this step, we also homogenize the names of the columns to make it easier to merge both datasets. In the end, we made a concatenation of rows.
4. Merge `violent_nuse` and `violent_siedco` in `crime` dataframe and delete duplicates using the algorithm developed in `1_clean_NUSE.ipynb`.

## 1. Import packages and data

In [24]:
# Beginning of code

# Load packages
import pandas as pd
import numpy as np
import re
from scipy.spatial import distance
from tqdm import tqdm

In [25]:
# Import violent_nuse
violent_nuse = pd.read_csv('../../Data/violent_nuse.csv', sep = ";")

In [26]:
# Import siedco
siedco = pd.read_csv('../../Data/siedco_raw.csv')

# Fix date format of FECHA_HECHO
siedco.FECHA_HECHO = pd.to_datetime(siedco.FECHA_HECHO)
# Filter to only have crimes from 2018
filtro = siedco.FECHA_HECHO.dt.year == 2018
siedco = siedco.loc[filtro,].reset_index(drop = True)

## 2. Filter to only have violent crimes in siedco

In [27]:
violent_crimes_typology = ['HURTO A COMERCIO', 'HURTO A PERSONAS', 
       'HURTO DE CELULARES', 'LESIONES PERSONALES', 'HURTO DE CELULARES (CASOS)', 
       'HURTO A RESIDENCIAS', 'DELITOS SEXUALES', 'HURTO DE BICICLETAS',
       'HOMICIDIOS', 'HURTO ABIGEATO', 'SECUESTRO', 'TERRORISMO', 
       'HURTO MOTOCICLETAS', 'HURTO AUTOMOTORES']
filtro = siedco.HECHO.isin(violent_crimes_typology)
violent_siedco = siedco.loc[filtro,].reset_index(drop = True)

## 3. Create a dictionary to homogenize the typologies of crimes

In [28]:
diccionario_delitos = {
    'HURTO A COMERCIO': '904 - HURTO EFECTUADO',
    'HURTO A PERSONAS': '904 - HURTO EFECTUADO',
    'HURTO DE CELULARES': '904 - HURTO EFECTUADO',
    'VIOLENCIA INTRAFAMILIAR': '906 - VIOLENCIA SEXUAL',
    'LESIONES PERSONALES': '910 - LESIONES PERSONALES',
    'HURTO DE CELULARES (CASOS)': '904 - HURTO EFECTUADO',
    'LESIONES EN AT': '910 - LESIONES PERSONALES',
    # siedco tipifica AMENAZAS dentro de "12 DELITOS CONTRA LA SEGURIDAD PÚBLICA"
    'AMENAZAS': '932 - ALTERACIÓN DEL ORDEN PÚBLICO',
    'HURTO A RESIDENCIAS': '904 - HURTO EFECTUADO',
    'DELITOS SEXUALES': '906 - VIOLENCIA SEXUAL',
    'HURTO DE BICICLETAS': '904 - HURTO EFECTUADO',
    'EXTORSION': '932 - ALTERACIÓN DEL ORDEN PÚBLICO',
    'HOMICIDIOS EN AT': '901 - MUERTO',
    'HOMICIDIOS': '901 - MUERTO',
    'HURTO A ENTIDADES FINANCIERAS': '904 - HURTO EFECTUADO',
    'HURTO PIRATERIA TERRESTRE': '904 - HURTO EFECTUADO',
    'HURTO ABIGEATO': '904 - HURTO EFECTUADO',
    'SECUESTRO': '903 - RAPTO / SECUESTRO', 
    'TERRORISMO': '932 - ALTERACIÓN DEL ORDEN PÚBLICO', 
    'HURTO MOTOCICLETAS': '904 - HURTO EFECTUADO',
    'HURTO AUTOMOTORES': '904 - HURTO EFECTUADO'}

In [29]:
# Create a new column in violent_siedco with the homogenized typologies of crime
violent_siedco["TIPO_DETALLE"] = violent_siedco["HECHO"].replace(diccionario_delitos)

In [30]:
# Homogenize column names
violent_siedco = violent_siedco[["TIPO_DETALLE", "FECHA_HECHO", "HORA", "LATITUD_Y", "LONGITUD_X"]]\
    .rename(columns = {"FECHA_HECHO": "FECHA", "LATITUD_Y": "LATITUD", "LONGITUD_X": "LONGITUD"})
violent_nuse = violent_nuse[['TIPO_DETALLE', 'FECHA', 'HORA', 'LATITUD', 'LONGITUD']]

In [31]:
# Create an id of the origin of the row
violent_siedco = violent_siedco.reset_index().rename(columns = {"index": "BASE"}).assign(BASE = lambda x: "siedco - " + x.BASE.astype(str))
violent_nuse = violent_nuse.reset_index().rename(columns = {"index": "BASE"}).assign(BASE = lambda x: "nuse - " + x.BASE.astype(str))

In [32]:
violent_siedco.to_csv('../../Data/violent_siedco.csv', sep = ";", index = False)

## 4. Merge and remove duplicates 

In [12]:
# Merge
crime = pd.concat([violent_siedco, violent_nuse]).reset_index(drop = True)

In [13]:
# Fix date format
crime.FECHA = pd.to_datetime(crime.FECHA)

In [15]:
# Create an id of the name of the original base
crime["BASE2"] = crime["BASE"].apply(lambda x: re.match("[A-Z]*", x)[0])

In [ ]:
# Delete duplicate reports

# An empty data frame to store the results
unique_crimes = pd.DataFrame()
# We iterate by dates
grilla_fechas = np.sort(crime.FECHA.unique())

for i in tqdm(grilla_fechas):
    # For each date, we identify all the crimes reported
    i = pd.to_datetime(i)
    filtro = crime.FECHA == i
    crime_small = crime.loc[filtro,].reset_index(drop = True)

    # Then, we divide the reports in groups according the typology of the crime
    tipos = crime_small["TIPO_DETALLE"].unique()
    for t in tipos:
        filtro = crime_small["TIPO_DETALLE"] == t
        crime_small2 = crime_small.loc[filtro,].reset_index(drop = True)

        # Now we are going to calculate the events that have happened in a distance less than 500 meters
        coords = crime_small2[["LATITUD", "LONGITUD"]].values
        # Firstly, calculate an euclidean distance between coords 
        eu_d = distance.cdist(coords, coords, 'euclidean')
        # Transform the results, that are in grades, to meters.
        # In the equator, one grade is equivalent to 111,319 meters
        dist_m = eu_d * 111319
        # Make the filter
        cercanos1 = dist_m < 500

        # Lastly, create a filter to identify if the distance between events is less than 8 hours
        horas = crime_small2["HORA"].values
        dist_h = np.abs(np.subtract.outer(horas, horas))
        cercanos2 = dist_h < 8

        # We define that we have the same crime if both the proximity condition of 500 meters and the 
        # proximity of 8 hours are met.
        cercanos = cercanos1 & cercanos2

        # Since the matrix is symmetric we only keep the upper triangular
        cercanos = np.triu(cercanos)

        # Now we are going to do the magic of only keeping the unique crimes. As our matrix is upper triangular, 
        # if we add by columns, those that are equal to 1 means that they are the only crimes
        indices_guardar = np.where(np.sum(cercanos, axis = 0) == 1)[0]
        
        # Store the results
        unique_crimes = pd.concat([unique_crimes, crime_small2.loc[indices_guardar,]]).reset_index(drop = True)

In [ ]:
unique_crimes.to_csv('../../Data/unique_crimes.csv', sep = ";", index = False)

In [ ]:
# End of the code